# Semi-supervision and domain adaptation with AdaMatch (AdaMatchを用いたセミスーパビジョンとドメインアダプテーション)

**Author:** [Sayak Paul](https://twitter.com/RisingSayak)<br>
**Date created:** 2021/06/19<br>
**Last modified:** 2021/06/19<br>
**Description:** Unifying semi-supervised learning and unsupervised domain adaptation with AdaMatch.

## 序章

この例では、[AdaMatch: A Unified Approach to Semi-Supervised Learning and Domain Adaptation](https://arxiv.org/abs/2106.04732)でBerthelotらによって提案されたAdaMatchアルゴリズムを実装します。このアルゴリズムは、教師なしドメイン適応の新たな最先端を示しています（2021年6月現在）。  
AdaMatchが特に興味深いのは、半教師付き学習(SSL)と教師なしドメイン適応(UDA)を一つのフレームワークで統合している点です。  
これにより、半教師付きドメイン適応（SSDA）を実行する方法を提供します。  
この例では、TensorFlow 2.5以上とTensorFlow Modelsが必要ですが、これらは以下のコマンドでインストールできます。

In [ ]:
!pip install -q tf-models-official

その前に、今回の例の基礎となる考え方を少しおさらいしておきましょう。

## プレリミナリー

**半教師付き学習(semi-supervised learning, SSL)** では、少量のラベル付きデータを用いて、より大きなラベル無しデータセットに対してモデルを学習します。コンピュータビジョンの半教師付き学習法としては、[FixMatch](https://arxiv.org/abs/2001.07685), [MixMatch](https://arxiv.org/abs/1905.02249), [Noisy Student Training](https://arxiv.org/abs/1911.04252)などが有名です。標準的なSSLのワークフローがどのようなものか、[this example](https://keras.io/examples/vision/consistency_training/)を参考にしてみてください。

**教師なしドメイン適応** では、ソースとなるラベル付きデータセットとターゲットとなる*ラベルなし*データセットにアクセスできます。タスクは、ターゲットデータセットにうまく一般化できるモデルを学習することである。ソースデータセットとターゲットデータセットは、分布の点で異なります。
次の図は、このアイデアを説明するものである。今回の例では、ソースデータセットとして[MNIST dataset](http://yann.lecun.com/exdb/mnist/)を使い、ターゲットデータセットは家の番号の画像で構成された[SVHN](http://ufldl.stanford.edu/housenumbers/)を使います。どちらのデータセットも、テクスチャ、視点、見た目など様々な要素が異なるため、ドメイン（分布）が互いに異なっています。

![](https://i.imgur.com/dJFSJuT.png)

深層学習でよく使われるドメイン適応アルゴリズムには、[Deep CORAL](https://arxiv.org/abs/1612.01939)、[Moment Matching](https://arxiv.org/abs/1812.01754)などがあります。

## 設定

In [ ]:
import tensorflow as tf

tf.random.set_seed(42)

import numpy as np

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from official.vision.image_classification.augment import RandAugment

import tensorflow_datasets as tfds

tfds.disable_progress_bar()

## データの準備

In [ ]:
# MNIST
(
    (mnist_x_train, mnist_y_train),
    (mnist_x_test, mnist_y_test),
) = keras.datasets.mnist.load_data()

# Add a channel dimension
mnist_x_train = tf.expand_dims(mnist_x_train, -1)
mnist_x_test = tf.expand_dims(mnist_x_test, -1)

# Convert the labels to one-hot encoded vectors
mnist_y_train = tf.one_hot(mnist_y_train, 10).numpy()

# SVHN
svhn_train, svhn_test = tfds.load(
    "svhn_cropped", split=["train", "test"], as_supervised=True
)

## 定数とハイパーパラメータの定義

In [ ]:
RESIZE_TO = 32

SOURCE_BATCH_SIZE = 64
TARGET_BATCH_SIZE = 3 * SOURCE_BATCH_SIZE  # Reference: Section 3.2
EPOCHS = 10
STEPS_PER_EPOCH = len(mnist_x_train) // SOURCE_BATCH_SIZE
TOTAL_STEPS = EPOCHS * STEPS_PER_EPOCH

AUTO = tf.data.AUTOTUNE
LEARNING_RATE = 0.03

WEIGHT_DECAY = 0.0005
INIT = "he_normal"
DEPTH = 28
WIDTH_MULT = 2

## データ拡張ユーティリティ

SSLアルゴリズムの標準的な要素は，学習モデルの予測に一貫性を持たせるために，同じ画像の弱増強版と強増強版を学習モデルに与えることである．強い
augmentationに対しては，[RandAugment](https://arxiv.org/abs/1909.13719)が標準的な選択です． 
weak augmentationでは、水平方向の反転とランダムクロッピングを使います。

In [ ]:
# Initialize `RandAugment` object with 2 layers of
# augmentation transforms and strength of 5.
augmenter = RandAugment(num_layers=2, magnitude=5)


def weak_augment(image, source=True):
    if image.dtype != tf.float32:
        image = tf.cast(image, tf.float32)

    # MNIST images are grayscale, this is why we first convert them to
    # RGB images.
    if source:
        image = tf.image.resize_with_pad(image, RESIZE_TO, RESIZE_TO)
        image = tf.tile(image, [1, 1, 3])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_crop(image, (RESIZE_TO, RESIZE_TO, 3))
    return image


def strong_augment(image, source=True):
    if image.dtype != tf.float32:
        image = tf.cast(image, tf.float32)

    if source:
        image = tf.image.resize_with_pad(image, RESIZE_TO, RESIZE_TO)
        image = tf.tile(image, [1, 1, 3])
    image = augmenter.distort(image)
    return image

## データ読み込みユーティリティ

In [ ]:
def create_individual_ds(ds, aug_func, source=True):
    if source:
        batch_size = SOURCE_BATCH_SIZE
    else:
        # During training 3x more target unlabeled samples are shown
        # to the model in AdaMatch (Section 3.2 of the paper).
        batch_size = TARGET_BATCH_SIZE
    ds = ds.shuffle(batch_size * 10, seed=42)

    if source:
        ds = ds.map(lambda x, y: (aug_func(x), y), num_parallel_calls=AUTO)
    else:
        ds = ds.map(lambda x, y: (aug_func(x, False), y), num_parallel_calls=AUTO)

    ds = ds.batch(batch_size).prefetch(AUTO)
    return ds

接尾辞の`_w`と`_s`は、それぞれ弱いことと強いことを表しています。

In [ ]:
source_ds = tf.data.Dataset.from_tensor_slices((mnist_x_train, mnist_y_train))
source_ds_w = create_individual_ds(source_ds, weak_augment)
source_ds_s = create_individual_ds(source_ds, strong_augment)
final_source_ds = tf.data.Dataset.zip((source_ds_w, source_ds_s))

target_ds_w = create_individual_ds(svhn_train, weak_augment, source=False)
target_ds_s = create_individual_ds(svhn_train, strong_augment, source=False)
final_target_ds = tf.data.Dataset.zip((target_ds_w, target_ds_s))

シングルイメージバッチのイメージはこんな感じです。

![](https://i.imgur.com/aver8cG.png)

## 損失計算ユーティリティ

In [ ]:
def compute_loss_source(source_labels, logits_source_w, logits_source_s):
    loss_func = keras.losses.CategoricalCrossentropy(from_logits=True)
    # First compute the losses between original source labels and
    # predictions made on the weakly and strongly augmented versions
    # of the same images.
    w_loss = loss_func(source_labels, logits_source_w)
    s_loss = loss_func(source_labels, logits_source_s)
    return w_loss + s_loss


def compute_loss_target(target_pseudo_labels_w, logits_target_s, mask):
    loss_func = keras.losses.CategoricalCrossentropy(from_logits=True, reduction="none")
    target_pseudo_labels_w = tf.stop_gradient(target_pseudo_labels_w)
    # For calculating loss for the target samples, we treat the pseudo labels
    # as the ground-truth. These are not considered during backpropagation
    # which is a standard SSL practice.
    target_loss = loss_func(target_pseudo_labels_w, logits_target_s)

    # More on `mask` later.
    mask = tf.cast(mask, target_loss.dtype)
    target_loss *= mask
    return tf.reduce_mean(target_loss, 0)

## AdaMatchトレーニング用にサブクラス化されたモデル

下図は、AdaMatchの全体的なワークフローを示しています（[原著論文](https://arxiv.org/abs/2106.04732)より引用）。

![](https://i.imgur.com/1QsEm2M.png)

ここでは、ワークフローを簡単にステップ・バイ・ステップで説明します。

1. まず，ソースデータセットとターゲットデータセットから，弱く増強された画像と強く増強された画像のペアを検索する．
2. 2つの連結されたコピーを用意する．
    i. 両方のペアが連結されたもの．
    ii. ソース・データ・イメージのペアのみが連結されたもの。
3. モデルに2つのフォワードパスを実行する。
    i. このフォワードパスでは、[Batch Normalization](https://arxiv.org/abs/1502.03167)の統計値が更新されます。
    ii. 2回目のフォワードパスでは、**2.ii**から得られた連結されたコピーのみを使用します。
    Batch Normalizationのレイヤーは推論モードで実行されます。
4. フォワードパスの両方について、それぞれのロジットを計算する。
5. ロジットは論文で紹介されている一連の変換を行う（これについては後述する）。
6. 損失を計算し，基礎となるモデルの勾配を更新する．

In [ ]:
class AdaMatch(keras.Model):
    def __init__(self, model, total_steps, tau=0.9):
        super(AdaMatch, self).__init__()
        self.model = model
        self.tau = tau  # Denotes the confidence threshold
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")
        self.total_steps = total_steps
        self.current_step = tf.Variable(0, dtype="int64")

    @property
    def metrics(self):
        return [self.loss_tracker]

    # This is a warmup schedule to update the weight of the
    # loss contributed by the target unlabeled samples. More
    # on this in the text.
    def compute_mu(self):
        pi = tf.constant(np.pi, dtype="float32")
        step = tf.cast(self.current_step, dtype="float32")
        return 0.5 - tf.cos(tf.math.minimum(pi, (2 * pi * step) / self.total_steps)) / 2

    def train_step(self, data):
        ## Unpack and organize the data ##
        source_ds, target_ds = data
        (source_w, source_labels), (source_s, _) = source_ds
        (
            (target_w, _),
            (target_s, _),
        ) = target_ds  # Notice that we are NOT using any labels here.

        combined_images = tf.concat([source_w, source_s, target_w, target_s], 0)
        combined_source = tf.concat([source_w, source_s], 0)

        total_source = tf.shape(combined_source)[0]
        total_target = tf.shape(tf.concat([target_w, target_s], 0))[0]

        with tf.GradientTape() as tape:
            ## Forward passes ##
            combined_logits = self.model(combined_images, training=True)
            z_d_prime_source = self.model(
                combined_source, training=False
            )  # No BatchNorm update.
            z_prime_source = combined_logits[:total_source]

            ## 1. Random logit interpolation for the source images ##
            lambd = tf.random.uniform((total_source, 10), 0, 1)
            final_source_logits = (lambd * z_prime_source) + (
                (1 - lambd) * z_d_prime_source
            )

            ## 2. Distribution alignment (only consider weakly augmented images) ##
            # Compute softmax for logits of the WEAKLY augmented SOURCE images.
            y_hat_source_w = tf.nn.softmax(final_source_logits[: tf.shape(source_w)[0]])

            # Extract logits for the WEAKLY augmented TARGET images and compute softmax.
            logits_target = combined_logits[total_source:]
            logits_target_w = logits_target[: tf.shape(target_w)[0]]
            y_hat_target_w = tf.nn.softmax(logits_target_w)

            # Align the target label distribution to that of the source.
            expectation_ratio = tf.reduce_mean(y_hat_source_w) / tf.reduce_mean(
                y_hat_target_w
            )
            y_tilde_target_w = tf.math.l2_normalize(
                y_hat_target_w * expectation_ratio, 1
            )

            ## 3. Relative confidence thresholding ##
            row_wise_max = tf.reduce_max(y_hat_source_w, axis=-1)
            final_sum = tf.reduce_mean(row_wise_max, 0)
            c_tau = self.tau * final_sum
            mask = tf.reduce_max(y_tilde_target_w, axis=-1) >= c_tau

            ## Compute losses (pay attention to the indexing) ##
            source_loss = compute_loss_source(
                source_labels,
                final_source_logits[: tf.shape(source_w)[0]],
                final_source_logits[tf.shape(source_w)[0] :],
            )
            target_loss = compute_loss_target(
                y_tilde_target_w, logits_target[tf.shape(target_w)[0] :], mask
            )

            t = self.compute_mu()  # Compute weight for the target loss
            total_loss = source_loss + (t * target_loss)
            self.current_step.assign_add(
                1
            )  # Update current training step for the scheduler

        gradients = tape.gradient(total_loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))

        self.loss_tracker.update_state(total_loss)
        return {"loss": self.loss_tracker.result()}

本論文では、3つの改善点を紹介しています。

* AdaMatchでは、2つのフォワードパスを実行し、そのうちの1つだけがBatch Normalization統計の更新を担当します。  
これは、ターゲットデータセットの分布の変化を考慮して行われます。  
もう一方のフォワードパスでは，ソースサンプルのみを使用し，バッチ正規化レイヤーは推論モードで実行されます。  
これらの2つのパスでのソース・サンプル（弱増強バージョンと強増強バージョン）のロジットは，バッチ・ノーマライゼーション・レイヤーがどのように実行されているかによって，互いに若干異なります。  
ソースサンプルの最終的なロジットは，これら2つの異なるロジットのペアの間を線形補間することで計算されます。  
これにより、整合性正則化の一形態が誘発されます。  
このステップは、 **ランダムロジット補間** と呼ばれています。

* ソースラベルとターゲットラベルの分布を揃えるために、 **分布調整** を行います。  
これにより、基礎となるモデルが「領域不変な表現」を学習するのに役立ちます。  
教師なしドメイン適応の場合、ターゲットデータセットのラベルにアクセスすることはできません。  
そのため、基礎モデルから擬似的なラベルが生成されます。

* 基礎モデルが対象サンプルの疑似ラベルを生成します。  
そのモデルが誤った予測をする可能性もあります。  
そのような予測は、学習が進むにつれて逆に伝播し、全体のパフォーマンスを低下させてしまいます。  
これを補うために，ある閾値に基づいて，信頼度の高い予測をフィルタリングします（そのため，`compute_loss_target()`の中で`mask`を使っています）。  
AdaMatchでは、このしきい値は相対的に調整されるので、**相対的信頼度しきい値**と呼ばれています。

これらの方法の詳細や、それぞれの方法がどのように貢献しているのかについては、[論文](https://arxiv.org/abs/2106.04732)を参照してください。

**`compute_mu()`について**:

AdaMatch では、固定のスカラー量を使用するのではなく、変化するスカラーを使用します。  
これは、ターゲットサンプルによって構成された損失の重みを表します。  
ウェイトスケジューラーは次のようになります。

![](https://i.imgur.com/dG7i9uH.png)

このスケジューラーは、トレーニングの前半では、目標とするドメインの損失の重みを0から1に増やし、後半ではその重みを1に保つ。  
そして、トレーニングの後半ではその重みを1のままにします。

## Wide-ResNet-28-2のインスタンス化

この例で使用しているデータセットペアには、著者は[WideResNet-28-2](https://arxiv.org/abs/1605.07146)を使用しています。  
以下のコードのほとんどは、[this script](https://github.com/asmith26/wide_resnets_keras/blob/master/main.py)を参考にしています。  
なお、以下のモデルは、ピクセル値を[0, 1]にスケーリングするスケーリングレイヤーを内蔵しています。

In [ ]:
def wide_basic(x, n_input_plane, n_output_plane, stride):
    conv_params = [[3, 3, stride, "same"], [3, 3, (1, 1), "same"]]

    n_bottleneck_plane = n_output_plane

    # Residual block
    for i, v in enumerate(conv_params):
        if i == 0:
            if n_input_plane != n_output_plane:
                x = layers.BatchNormalization()(x)
                x = layers.Activation("relu")(x)
                convs = x
            else:
                convs = layers.BatchNormalization()(x)
                convs = layers.Activation("relu")(convs)
            convs = layers.Conv2D(
                n_bottleneck_plane,
                (v[0], v[1]),
                strides=v[2],
                padding=v[3],
                kernel_initializer=INIT,
                kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
                use_bias=False,
            )(convs)
        else:
            convs = layers.BatchNormalization()(convs)
            convs = layers.Activation("relu")(convs)
            convs = layers.Conv2D(
                n_bottleneck_plane,
                (v[0], v[1]),
                strides=v[2],
                padding=v[3],
                kernel_initializer=INIT,
                kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
                use_bias=False,
            )(convs)

    # Shortcut connection: identity function or 1x1
    # convolutional
    #  (depends on difference between input & output shape - this
    #   corresponds to whether we are using the first block in
    #   each
    #   group; see `block_series()`).
    if n_input_plane != n_output_plane:
        shortcut = layers.Conv2D(
            n_output_plane,
            (1, 1),
            strides=stride,
            padding="same",
            kernel_initializer=INIT,
            kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
            use_bias=False,
        )(x)
    else:
        shortcut = x

    return layers.Add()([convs, shortcut])


# Stacking residual units on the same stage
def block_series(x, n_input_plane, n_output_plane, count, stride):
    x = wide_basic(x, n_input_plane, n_output_plane, stride)
    for i in range(2, int(count + 1)):
        x = wide_basic(x, n_output_plane, n_output_plane, stride=1)
    return x


def get_network(image_size=32, num_classes=10):
    n = (DEPTH - 4) / 6
    n_stages = [16, 16 * WIDTH_MULT, 32 * WIDTH_MULT, 64 * WIDTH_MULT]

    inputs = keras.Input(shape=(image_size, image_size, 3))
    x = layers.experimental.preprocessing.Rescaling(scale=1.0 / 255)(inputs)

    conv1 = layers.Conv2D(
        n_stages[0],
        (3, 3),
        strides=1,
        padding="same",
        kernel_initializer=INIT,
        kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
        use_bias=False,
    )(x)

    ## Add wide residual blocks ##

    conv2 = block_series(
        conv1,
        n_input_plane=n_stages[0],
        n_output_plane=n_stages[1],
        count=n,
        stride=(1, 1),
    )  # Stage 1

    conv3 = block_series(
        conv2,
        n_input_plane=n_stages[1],
        n_output_plane=n_stages[2],
        count=n,
        stride=(2, 2),
    )  # Stage 2

    conv4 = block_series(
        conv3,
        n_input_plane=n_stages[2],
        n_output_plane=n_stages[3],
        count=n,
        stride=(2, 2),
    )  # Stage 3

    batch_norm = layers.BatchNormalization()(conv4)
    relu = layers.Activation("relu")(batch_norm)

    # Classifier
    trunk_outputs = layers.GlobalAveragePooling2D()(relu)
    outputs = layers.Dense(
        num_classes, kernel_regularizer=regularizers.l2(WEIGHT_DECAY)
    )(trunk_outputs)

    return keras.Model(inputs, outputs)

これで、Wide ResNetモデルを以下のようにインスタンス化することができます。なお、ここでWide ResNetを使う目的は、実装をできるだけオリジナルに近づけるためです。

In [ ]:
wrn_model = get_network()
print(f"Model has {wrn_model.count_params()/1e6} Million parameters.")

## AdaMatchモデルのインスタンス化とコンパイル

In [ ]:
reduce_lr = keras.experimental.CosineDecay(LEARNING_RATE, TOTAL_STEPS, 0.25)
optimizer = keras.optimizers.Adam(reduce_lr)

adamatch_trainer = AdaMatch(model=wrn_model, total_steps=TOTAL_STEPS)
adamatch_trainer.compile(optimizer=optimizer)

## モデルトレーニング

In [ ]:
total_ds = tf.data.Dataset.zip((final_source_ds, final_target_ds))
adamatch_trainer.fit(total_ds, epochs=EPOCHS)

## ターゲットテストセットとソーステストセットでの評価

In [ ]:
# Compile the AdaMatch model to yield accuracy.
adamatch_trained_model = adamatch_trainer.model
adamatch_trained_model.compile(metrics=keras.metrics.SparseCategoricalAccuracy())

# Score on the target test set.
svhn_test = svhn_test.batch(TARGET_BATCH_SIZE).prefetch(AUTO)
_, accuracy = adamatch_trained_model.evaluate(svhn_test)
print(f"Accuracy on target test set: {accuracy * 100:.2f}%")

トレーニングを重ねることで、このスコアは向上します。同じネットワークを標準的な分類目的で学習した場合、精度は **7.20%** となり、AdaMatchで得られたものよりも著しく低いものとなります。 
ハイパーパラメータやその他の実験の詳細については、[this notebook](https://colab.research.google.com/github/sayakpaul/AdaMatch-TF/blob/main/Vanilla_WideResNet.ipynb)をご覧ください

In [ ]:
# Utility function for preprocessing the source test set.
def prepare_test_ds_source(image, label):
    image = tf.image.resize_with_pad(image, RESIZE_TO, RESIZE_TO)
    image = tf.tile(image, [1, 1, 3])
    return image, label


source_test_ds = tf.data.Dataset.from_tensor_slices((mnist_x_test, mnist_y_test))
source_test_ds = (
    source_test_ds.map(prepare_test_ds_source, num_parallel_calls=AUTO)
    .batch(TARGET_BATCH_SIZE)
    .prefetch(AUTO)
)

# Evaluation on the source test set.
_, accuracy = adamatch_trained_model.evaluate(source_test_ds)
print(f"Accuracy on source test set: {accuracy * 100:.2f}%")

この[モデルウェイト](https://github.com/sayakpaul/AdaMatch-TF/releases/tag/v1.0.0)を使って再現することができます。